# *KAGGLE CHALLENGE: LANL Earthquake Prediction*

Un projet de Matthieu Dagommer, Paul Boulgakoff, Godefroy Bichon, Germain L'Hostis

Versions utilisées:

Python: 3.10.4
Torch: 1.11

In [1]:
import numpy as np
import math
import matplotlib
import matplotlib.pyplot as plt 
import time

import torch as th
th.cuda.empty_cache()

import torch.autograd as autograd
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn

from torchsummary import summary 

import gzip
import pickle

import pandas as pd
import scipy
import scipy.stats as stats
from scipy.stats import kurtosis, skew
import csv
import os
import pickle
import gc

from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [2]:
### Ensuring CPU is connected

print(th.cuda.is_available())
print(th.cuda.get_device_name())
device = th.device('cuda' if th.cuda.is_available() else 'cpu')

True
GeForce GTX 1050 Ti with Max-Q Design


# *General Hyperparameters*

In [3]:
### Setting General Hyperparameters 

batch_size = 100 # number of patches per batch
valid_rate = 0.1 # fraction of data dedicated to validation
overlap_rate = 0.2 # overlap

# Parameters

nrows = 200_000_000
seed = 1

In [4]:
th.manual_seed(seed)
np.random.seed(seed)

# *Data Preparation*



In [5]:
### Loading Data

rootpath = os.getcwd() + "/"

train_data = pd.read_csv(rootpath + "train.csv", usecols = ['acoustic_data', 'time_to_failure'], \
                         dtype = {'acoustic_data': np.int16, 'time_to_failure': np.float64}, nrows = nrows)

In [6]:
X = th.squeeze(th.tensor(train_data['acoustic_data'].values, dtype = th.int16))
Y = th.squeeze(th.tensor(train_data['time_to_failure'].values, dtype = th.float64))

del train_data
gc.collect()

0

In [7]:
### Retrieve Test Data

path = rootpath + "test/" + "seg_00030f.csv"
test = pd.read_csv(path)
patch_size = test.shape[0]

#path = rootpath + "sample_submission.csv"
#sample_submission = pd.read_csv(path)

#test_ttf_counts = sample_submission.time_to_failure.value_counts()

#X_test = []

#for filename in os.listdir(rootpath + "test"):
#    temp_df = pd.read_csv(rootpath + "test/" + filename)
#    X_test.append(temp_df)

#Xtest = th.zeros((len(X_test),time_window))
#for i in range(len(X_test)):
#  Xtest[i,:] = th.tensor(X_test[i]["acoustic_data"], dtype = th.float32)

#Ytest = th.tensor(sample_submission["time_to_failure"])

In [8]:
### Function to create patched sequences with some overlap out of the training data

# Overlap is expressed as a fraction of patch size
def patching(patch_size, X, Y, overlap_rate):
    
    overlap = int(patch_size*overlap_rate)
    L = X.shape[0] # total length of the training acoustic signal
    n_patch = int(np.floor((L-patch_size)/(patch_size-overlap)+1))
    ids_no_seism = []

    X_patch = th.zeros(n_patch,patch_size)
    Y_patch = th.zeros(n_patch)
    
    for i in range (n_patch):
        X_patch[i,:] = X[i*(patch_size-overlap):(i+1)*patch_size-i*overlap] 
        Y_patch[i] = Y[(i+1)*patch_size - i*overlap] 
    
        # Removing patches with no seism
        if th.min(Y[i*(patch_size-overlap):(i+1)*patch_size - i*overlap]) > 0.001:
            ids_no_seism.append(i)
    
    X_patch = X_patch[ids_no_seism]
    Y_patch = Y_patch[ids_no_seism]
    
    return(n_patch, X_patch, Y_patch)

In [9]:
### Normalizing Data

ss = StandardScaler()
mm = MinMaxScaler()
    
X = th.squeeze(th.from_numpy(ss.fit_transform(np.array(X).reshape(-1, 1))))
Y = th.squeeze(th.from_numpy(mm.fit_transform(np.array(Y).reshape(-1, 1))))

In [10]:
### Distribution of data in patches

_, X_patch, Y_patch = patching(patch_size, X, Y, overlap_rate = overlap_rate)

del X; del Y
gc.collect()

print(X_patch.shape)
print("There are ", X_patch.shape[0], "time series available for training and validation after patching with overlap. \n")

torch.Size([1656, 150000])
There are  1656 time series available for training and validation after patching with overlap. 



In [11]:
### Initializating Data Sets


# Shuffling data
idx = np.arange(X_patch.shape[0])
shuffled_idx = np.random.shuffle(idx)
X_patch = th.squeeze(X_patch[shuffled_idx,:])
Y_patch = th.squeeze(Y_patch[shuffled_idx])

N_samples = X_patch.shape[0]
N_valid = int(valid_rate*N_samples) # number of validation patches

X_valid = X_patch[:N_valid,:].cpu()
Y_valid = Y_patch[:N_valid].cpu()


# Inputs of nn.Conv1d must have the following shape: (N, C_in, *),
#where N: number of samples (batch size), C_in: number of input channels, *: can be any dimension (150_000 for our time series)

# Add channel dimension
X_valid = th.unsqueeze(X_valid, 1)
X_train = X_patch[N_valid:,:].cpu()
Y_train = Y_patch[N_valid:].cpu()
N_train = X_train.shape[0]

n_batch = int(X_train.shape[0] / batch_size) # Number of batch per epoch

X_train = X_train.reshape([N_train, 1, patch_size])

# *Function for Results plotting*

In [12]:
### Plotting Graphs

def plot_and_save_results(train_losses, valid_losses, best_mvd, best_mtd, min_tl, min_vl, mm, model_name):

    best_epoch = np.fromiter(valid_losses, dtype=float).argmin()
    
    fig, ax = plt.subplots(1, 2, figsize = (20,10))
    plt.rcParams['font.size'] = '20'
    ax[0].set(title = "Losses: Training and Validation") 
    ax[0].set_xlabel("epochs", fontsize = 20)
    ax[0].set_ylabel("MSE", fontsize = 20)
    ax[0].plot(train_losses,"r", label = "Training", linewidth = 3)
    ax[0].plot(valid_losses, "b", label = "Validation", linewidth = 3)
    ax[0].legend(loc = "upper right", fontsize = 18)
    ax[0].axvline(x=int(best_epoch), color = 'black', linestyle ="--", linewidth = 3)
    ax[0].annotate("Best epoch: {}\nMSE_train: {:.3f}\nMSE_valid: {:.3f}".format(int(best_epoch), min_tl, min_vl), \
                   xy = (0.5,0.5), xycoords = 'axes fraction')
    
    best_mvd_plot = mm.inverse_transform(best_mvd.reshape(-1, 1))
    best_mtd_plot = mm.inverse_transform(best_mtd.reshape(-1, 1))

    N_valid = best_mvd_plot.shape[0]
    
    mean = float(np.mean(best_mvd_plot))
    std_dev = float(np.std(best_mvd_plot))
    kurt = float(kurtosis(best_mvd_plot))
    skewn = float(skew(best_mvd_plot))
    q1 = float(np.quantile(best_mvd_plot, 0.25))
    median = float(np.quantile(best_mvd_plot, 0.5))
    q3 = float(np.quantile(best_mvd_plot, 0.75))
    mae = np.absolute(best_mvd_plot).sum() / N_valid
    mse = np.square(best_mvd_plot).sum() / N_valid
    
    text = "mean: {:.3f}\nstd: {:.3f}\nkurt: {:.3f}\nskew: {:.3f}\nq1: {:.3f}\nmed: {:.3f}\nq3: {:.3f}\niqr: {:.3f}\nmae: {:.3f}\nmse: {:.3f}\n".format(mean, std_dev, kurt, skewn, q1, median, q3, q3-q1, mae, mse)
    
    ax[1].hist(best_mvd_plot, alpha = 0.3, label = "validation set", bins = 100, density = True, range = (-16, 16))
    ax[1].set(title = "TTF error distributions at best epoch")
    ax[1].set_xlabel("Error (seconds)", fontsize = 20)
    ax[1].set_ylabel("Density", fontsize = 20)
    ax[1].hist(best_mtd_plot, alpha = 0.3, label = "training set", bins = 100, density = True, range = (-16, 16))
    ax[1].annotate(text, xy =(-15, 0.1))
    ax[1].legend()


    plt.gcf()
    plt.savefig('Models/' + model_name + '/' + model_name + "_plot.jpg")
    plt.show()

    model_features = {"N_samples": N_samples, "N_train": N_train, "N_valid": N_valid, \
                      "overlap_rate": overlap_rate, "learning_rate": learning_rate, "num_epochs": num_epochs, \
                     "seed": seed, "batch_size": batch_size, "train_losses": train_losses, "valid_losses": valid_losses, \
                     "valid_differences": valid_differences, "best_mtd": best_mtd, "best_mvd": best_mvd, "min_tl": min_tl, \
                      "min_vl": min_vl}

    pickle.dump(model_features, open('Models/' + model_name + '/' + model_name + ".p", "wb" ))
    
    model_features_display = {"N_samples": N_samples, "N_train": N_train, "N_valid": N_valid, \
                  "overlap_rate": overlap_rate, "learning_rate": learning_rate, "num_epochs": num_epochs, \
                 "seed": seed, "batch_size": batch_size}
    
    pickle.dump(model_features_display, open('Models/' + model_name + '/' + model_name + "display.p", "wb" ))

# *1D Convolutional Neural Network*

This section contains two 1D CNN architecture that we designed during this project, with their respective hyperparameters and the training loop.
To use the LSTM instead, you can skip this section and go to the next.

In [ ]:
### Hyperparameters specific to 1D CNN

learning_rate=0.0001
num_epochs=50


In [ ]:
### 1D Convolutional Network #1

class NN(nn.Module):
    
    def __init__ (self,num_classes=1):
        super(NN, self).__init__()
        self.fc1=nn.Conv1d(in_channels=1, out_channels=1, 
                             kernel_size=10, padding=1, stride=5)
        self.fc2=nn.AdaptiveMaxPool1d(200)
        self.fct3=nn.Linear(200,50)
        self.fct4=nn.Linear(50,num_classes)
    
    def forward (self,x):
        x=self.fc1(x)
        x=self.fc2(x)
        x=self.fct3(x)
        x=self.fct4(x)
        return x
    

In [ ]:
### 1D Convolutional Network #2

class NN_2(nn.Module):
    def __init__(self,num_classes=1):
        super(NN_2, self).__init__()
        self.seq_1=nn.Sequential(nn.Conv1d(in_channels = 1, out_channels = 16, kernel_size = 10, stride = 5),
                                 nn.ReLU(),
                                 nn.MaxPool1d(kernel_size = 2),
                                 nn.Conv1d(in_channels = 16, out_channels = 32, kernel_size = 10, stride = 5, \
                                           padding = 'valid'),
                                 nn.ReLU(),
                                 nn.MaxPool1d(kernel_size = 2),
                                 nn.Conv1d(in_channels = 32, out_channels = 64, kernel_size = 10, stride = 5, \
                                           padding = 'valid'),
                                 nn.ReLU()
                                )
        
        self.seq_2=nn.Sequential(nn.Dropout(p = 0.4),
                                 nn.Linear(in_features = 64, out_features = 1)
                                )
    def forward(self,x):
        x=self.seq_1(x)
        x = th.mean(x, dim = 2, keepdim = True)
        x = th.squeeze(x)
        x=self.seq_2(x)
        return (x)

In [ ]:
### Initialize Network, Define Loss and optimizer

model = NN_2()
#print(model)
model.to(device)
loss=nn.MSELoss()
optimizer=th.optim.Adam(model.parameters(),learning_rate)


In [ ]:
### Plot Model Graph 

#from torchviz import make_dot
#yhat = model(X_valid.to(device))
#make_dot(yhat, params=dict(list(model.named_parameters()))).render("cnn_2", format="png")

In [ ]:
### Check accuracy during training

def check_accuracy(X_valid, Y_valid, model):
    
    device = 'cuda'
    N_valid = X_valid.shape[0]
    real_ttf = th.squeeze(Y_valid)
    loss_fn = nn.MSELoss()
    
    with th.no_grad():

        scores = th.squeeze(model(X_valid.to(device))).cpu()
        valid_loss_ = loss_fn(scores, real_ttf).cpu()
        valid_differences = scores[:] - real_ttf[:]
        valid_differences = valid_differences.cpu().numpy()
        valid_loss = valid_loss_.item()
    
    return valid_loss, valid_differences

In [ ]:
### Train network

th.cuda.empty_cache()

train_losses, valid_losses = [], []

best_mvd, best_mtd = [], [] # validation, training differences at best epoch

min_vl = 1000
min_tl = 1000

N_train_per_epoch = n_batch*batch_size 

start = time.perf_counter()

for epoch in range(num_epochs):

    th.cuda.empty_cache()
    
    # Generating Random Batches every epoch
    random_idx = np.random.randint(0, N_train, N_train_per_epoch)
    X_train_batch = th.reshape(X_train[random_idx], [n_batch, batch_size, 1, patch_size])
    Y_train_batch = th.reshape(Y_train[random_idx], [n_batch, batch_size, 1, 1])
    
    running_loss = 0
    _loss = 0

    for ids in range (n_batch):
        
        # Set gradients to 0
        optimizer.zero_grad()

        # Model Prediction
        predicted_ttfs = th.squeeze(model(X_train_batch[ids].cuda())).cpu()
        # Loss function
        _loss = loss(predicted_ttfs, th.squeeze(Y_train_batch[ids]))
        
        del predicted_ttfs
        gc.collect()
        
        # Backpropagation
        _loss.backward()
        running_loss += _loss.item()
        
        # Updating Model Weights
        optimizer.step()
        
    optimizer.zero_grad()
    th.cuda.empty_cache()
    
    model.eval()
    with th.no_grad():
    
        # Check Accuracy with Validation Set
        
        Y_valid_pred = th.squeeze(model(X_valid.cuda())).cpu()
        valid_loss = loss(Y_valid_pred, Y_valid)
        valid_differences = Y_valid_pred[:] - Y_valid[:]
        valid_differences = valid_differences.numpy()
        
        valid_losses.append(valid_loss.item())
        train_losses.append(running_loss / n_batch)
        
        if valid_loss < min_vl:
            
            min_vl = valid_loss
            min_tl = running_loss
            
            best_mvd = valid_differences
            
            Y_final = th.squeeze(model(X_train[:int(N_train/10)].to(device))).cpu()
            
            best_mtd = Y_train[:int(N_train/10)] - Y_final[:]
            best_mtd = best_mtd.cpu().detach().numpy()

    model.train()
    
    if epoch%10 == 0:
        print("Epoch: {}\t".format(epoch),
                "train Loss: {:.5f}.. ".format(train_losses[-1]),
                "valid Loss: {:.5f}.. ".format(valid_losses[-1])) 

print("---------- Best : {:.3f}".format(min(valid_losses)), " at epoch " 
    , np.fromiter(valid_losses, dtype=float).argmin(), " / ", epoch + 1)


end = time.perf_counter()
print("\ntime elapsed: {:.3f}".format(end - start))

In [ ]:
model_name = input("Choose a name for the model: ")
os.mkdir('Models/' + model_name)

In [ ]:
plot_and_save_results(train_losses, valid_losses, best_mvd, best_mtd, min_tl, min_vl, mm, model_name)

# *LSTM*

LSTM is a type of recurrent neural network and is a relevant architecture to treat temporal sequences. 

Pytorch Notes:

"Before getting to the example, note a few things. Pytorch’s LSTM expects all of its inputs to be 3D tensors. The semantics of the axes of these tensors is important. The first axis is the sequence itself, the second indexes instances in the mini-batch, and the third indexes elements of the input."

https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html


In [13]:
### Hyperparameters specific to LSTM

num_epochs = 1000 #1000 epochs
learning_rate = 0.001 #0.001 lr
hidden_size = 1 #number of features in hidden state
num_layers = 1 #number of stacked lstm layers => should stay at 1 unless you want to combine two LSTMs together
N_sub_patches = 250
N_features = 10
device = "cuda"
n_batch = 1 # One batch with all training data

In [14]:
def rearranging_X(X, N_sub_patches, L_sub_patch):
    
    #print(type(X))
    N_samples = X.shape[0]
    x = X.reshape(N_samples, N_sub_patches, L_sub_patch)
    x_mean = np.mean(x, axis = 2).reshape(N_samples, N_sub_patches, 1)
    x_std = np.std(x, axis = 2).reshape(N_samples, N_sub_patches, 1)
    x_skew = np.array(scipy.stats.skew(x, axis = 2), dtype = np.double).reshape(N_samples, N_sub_patches, 1)
    x_kurt = np.array(scipy.stats.kurtosis(x, axis = 2), dtype = np.double).reshape(N_samples, N_sub_patches, 1)
    x_min = np.min(x, axis = 2).reshape(N_samples, N_sub_patches, 1)
    x_max = np.max(x, axis = 2).reshape(N_samples, N_sub_patches, 1)
    x_q1 = np.quantile(x, 0.25, axis = 2).reshape(N_samples, N_sub_patches, 1)
    x_med = np.quantile(x, 0.5, axis = 2).reshape(N_samples, N_sub_patches, 1)
    x_q3 = np.quantile(x, 0.75, axis = 2).reshape(N_samples, N_sub_patches, 1)
    x_iqr = x_q3 - x_q1
    
    X_rearranged = np.concatenate((x_mean, x_std, x_skew, x_kurt, x_min, x_max, x_q1, x_med, x_q3, x_iqr), axis = 2)
    return X_rearranged
    

def lstm_feature_engineering(X_train, Y_train, X_valid, Y_valid, patch_size, N_sub_patches):

    X_train = np.array(th.squeeze(X_train)); Y_train = np.array(Y_train)
    X_valid = np.array(th.squeeze(X_valid)); Y_valid = np.array(Y_valid)

    L_seq = X_train.shape[1]
    L_sub_patch = int(L_seq / N_sub_patches)
    N_features = 10 # mean, std, skew, kurt, min, max, quantiles 0.25, 0.5, 0.75, inter-quartile range
    
    X_train_rearranged = rearranging_X(X_train, N_sub_patches, L_sub_patch)
    X_valid_rearranged = rearranging_X(X_valid, N_sub_patches, L_sub_patch)

    X_train = th.from_numpy(X_train_rearranged)
    X_valid = th.from_numpy(X_valid_rearranged)

    Y_train = th.from_numpy(Y_train); Y_valid = th.from_numpy(Y_valid)
    #Y_train = th.unsqueeze(Y_train, -1); Y_valid = th.unsqueeze(Y_valid, -1)

    return X_train, Y_train, X_valid, Y_valid

In [15]:
X_train_, Y_train_, X_valid_, Y_valid_ = lstm_feature_engineering(X_train, Y_train, X_valid, Y_valid, patch_size, N_sub_patches = N_sub_patches)
N_features = X_train_.shape[-1]

In [16]:
#Inspired from https://cnvrg.io/pytorch-lstm/

class Lstm(nn.Module):
    
    def __init__(self, N_features, hidden_size, num_layers, seq_length):
        super(Lstm, self).__init__()

        self.num_layers = num_layers # number of layers
        self.N_features = N_features # number of features
        self.hidden_size = hidden_size # hidden state
        self.seq_length = seq_length # sequence length

        self.lstm = nn.LSTM(input_size=N_features, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True) #lstm => Input Shape : (N_batch, L_seq, N_feature)
    
    def forward(self,x):
        
        #h_0 = th.zeros(self.num_layers, x.size(0), self.hidden_size) #hidden state
        #c_0 = th.zeros(self.num_layers, x.size(0), self.hidden_size) #internal state
        # Propagate input through LSTM
        #output, (hn, cn) = self.lstm(x.float(), (h_0, c_0)) #lstm with input, hidden, and internal state
        #output, (hn, cn) = self.lstm(x.float()) #lstm with input
        output, (_,_) = self.lstm(x.float()) #lstm with input
        #hn = hn.view(-1, self.hidden_size)  # reshaping the data for Dense layer next.
                                            # the size -1 is inferred from other dimensions
        #out = hn
        # output has the shape (N, seq_length, 1)
        #out = th.squeeze(output, -1)
        out = output[:,-1,0] # Retrieving predicted time at the end of the training
        
        return out

In [17]:
L_seq = N_sub_patches

In [18]:
model = Lstm(N_features, hidden_size, num_layers, L_seq) #our lstm class
model.cuda()

Lstm(
  (lstm): LSTM(10, 1, batch_first=True)
)

In [19]:
loss = th.nn.MSELoss()    # mean-squared error for regression
optimizer = th.optim.Adam(model.parameters(), lr=learning_rate) 

In [22]:
# Training LSTM

th.cuda.empty_cache()

train_losses, valid_losses = [], []
best_mvd, best_mtd = [], []

min_vl = 1000
min_tl = 1000

start = time.perf_counter()

for epoch in range(num_epochs):
    
    outputs = model.forward(X_train_.cuda()) #forward pass
    optimizer.zero_grad() #calculate the gradient, manually setting to 0
    # obtain the loss function
    _loss = loss(outputs, Y_train_.cuda())
    _loss.backward() #calculates the loss of the loss function
    running_loss = _loss.item()
    optimizer.step() #improve from loss, i.e backprop


    model.eval()
    with th.no_grad():
        
        Y_valid_pred = th.squeeze(model(X_valid_.cuda())).cpu()
        valid_loss = loss(Y_valid_pred, Y_valid_)
        valid_differences = Y_valid_pred[:] - Y_valid_[:]
        valid_differences = valid_differences.numpy()
        
        valid_losses.append(valid_loss.item())
        train_losses.append(running_loss / n_batch)
        
        if valid_loss < min_vl:

            min_vl = valid_loss
            min_tl = running_loss

            best_mvd = valid_differences

            Y_final = th.squeeze(model(X_train_.cuda())).cpu()

            best_mtd = Y_train_ - Y_final
            best_mtd = best_mtd.cpu().detach().numpy()
            
    model.train() 
    
    if epoch%10 == 0:
        print("Epoch: {}\t".format(epoch),
                "train Loss: {:.5f}.. ".format(train_losses[-1]),
                "valid Loss: {:.5f}.. ".format(valid_losses[-1])) 
        
print("---------- Best : {:.3f}".format(min(valid_losses)), " at epoch " 
    , np.fromiter(valid_losses, dtype=float).argmin(), " / ", epoch + 1)
    
end = time.perf_counter()
print("\ntime elapsed: {:.3f}".format(end - start))

Epoch: 0	 train Loss: 0.91807..  valid Loss: 1.10350.. 
Epoch: 10	 train Loss: 0.87599..  valid Loss: 1.05680.. 
Epoch: 20	 train Loss: 0.83104..  valid Loss: 1.00782.. 
Epoch: 30	 train Loss: 0.78497..  valid Loss: 0.95279.. 
Epoch: 40	 train Loss: 0.73718..  valid Loss: 0.89875.. 
Epoch: 50	 train Loss: 0.68942..  valid Loss: 0.84610.. 
Epoch: 60	 train Loss: 0.64310..  valid Loss: 0.79642.. 
Epoch: 70	 train Loss: 0.59853..  valid Loss: 0.75052.. 
Epoch: 80	 train Loss: 0.55639..  valid Loss: 0.70860.. 
Epoch: 90	 train Loss: 0.51767..  valid Loss: 0.67017.. 
Epoch: 100	 train Loss: 0.48204..  valid Loss: 0.63451.. 
Epoch: 110	 train Loss: 0.44913..  valid Loss: 0.60116.. 
Epoch: 120	 train Loss: 0.41868..  valid Loss: 0.56993.. 
Epoch: 130	 train Loss: 0.39056..  valid Loss: 0.54080.. 
Epoch: 140	 train Loss: 0.36469..  valid Loss: 0.51375.. 
Epoch: 150	 train Loss: 0.34099..  valid Loss: 0.48873.. 
Epoch: 160	 train Loss: 0.31938..  valid Loss: 0.46559.. 
Epoch: 170	 train Loss: 0

In [ ]:
model_name = input("Choose a name for the model: ")
os.mkdir('Models/' + model_name)

In [ ]:
plot_and_save_results(train_losses, valid_losses, best_mvd, best_mtd, min_tl, min_vl, mm, model_name)

In [ ]:
model_features = pickle.load(open(model_name + "display.p", "rb" ))
print(model_features)